<a href="https://colab.research.google.com/github/benihime91/pytorch_retinanet/blob/master/nbs/pascal_2007.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# What GPU do we have ?
! nvidia-smi

**Standard imports & setup**

In [ ]:
# Ensure colab doesn't disconnect
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

In [ ]:
# install dependencies
! pip install pytorch-lightning wandb
! pip install git+https://github.com/albumentations-team/albumentations

In [ ]:
# Grab the Data
! unzip -qq /content/drive/My\ Drive/Pascal\ 2007\ Data/pascal_voc_2007_test.zip
! unzip -qq /content/drive/My\ Drive/Pascal\ 2007\ Data/pascal_voc_2007_train_val.zip

In [ ]:
# Clone the RetinaNet Repo:
! git clone https://github.com/benihime91/pytorch_retinanet.git

In [ ]:
import os
import sys
import warnings

warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# use wandb to track experiments : Comment this if not using wandb logger
! wanbd login # a74f67fd5fae293e301ea8b6710ee0241f595a63

In [ ]:
from typing import *
import pandas as pd
import numpy as np
import re
import time
import pickle
import argparse

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    LearningRateLogger,
)

from pytorch_retinanet.src.models import Retinanet
from pytorch_retinanet.src.utils.eval_utils import CocoEvaluator
from pytorch_retinanet.src.utils.eval_utils import get_coco_api_from_dataset
from pytorch_retinanet.src.utils.general_utils import collate_fn
from pytorch_retinanet import DetectionDataset, Visualizer

pl.seed_everything(123)
pd.set_option("display.max_colwidth", None)

**Load in the Data:**  


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/gist/benihime91/bad475a40d16add314ba7be407803940/preprocess.ipynb) 
[preprocess_pascal.ipynb](https://github.com/benihime91/pytorch_retinanet/blob/master/nbs/preprocess_pascal.ipynb) : notebook to preprocess the `PascalVOC2007`data.

In [ ]:
trn_df = pd.read_csv('/content/drive/My Drive/Pascal 2007 Data/trn_data.csv')
val_df = pd.read_csv('/content/drive/My Drive/Pascal 2007 Data/val_data.csv')
tst_df = pd.read_csv('/content/drive/My Drive/Pascal 2007 Data/tst_data.csv')

# Load in the Label Dict
label_dict = pickle.load(open("/content/drive/My Drive/Pascal 2007 Data/names.pkl", "rb"))

In [ ]:
trn_df.head() # train dataframe

In [ ]:
val_df.head() # validation dataframe

In [ ]:
tst_df.head() # test dataframe

In [ ]:
label_dict # a dictionary which stores the mapping of target_labels to class_labels

In [ ]:
# Instantiate the visualizer
viz = Visualizer(class_names=label_dict)

def display_random_image(df: pd.DataFrame) -> None:
    "displays a radom Image from given dataframe"
    n = np.random.randint(0, len(df))
    fname = df["filename"][n]
    boxes = df.loc[df["filename"] == fname][["xmin", "ymin", "xmax", "ymax"]].values
    labels = df.loc[df["filename"] == fname]["labels"].values
    viz.draw_bboxes(fname, boxes=boxes, classes=labels, figsize=(10, 10))

In [ ]:
# Display random Image from the train set
display_random_image(trn_df)
display_random_image(trn_df)

In [ ]:
# Display random Image from the validation set
display_random_image(val_df)
display_random_image(val_df)

In [ ]:
# Display random Image from the Test Dataset
display_random_image(tst_df)
display_random_image(tst_df)

**Instantiate `transforms`:**

In [ ]:
train_transformations = [
    A.HorizontalFlip(p=0.5),
    A.RandomSizedBBoxSafeCrop(600, 600, erosion_rate=0.2, p=0.3),
    A.ToFloat(max_value=255.0, always_apply=True),
    ToTensorV2(always_apply=True),
]


valid_transformations = [
    A.ToFloat(max_value=255.0, always_apply=True),
    ToTensorV2(always_apply=True),
]

# Transformations for train dataset
trn_tfms = A.Compose(
    train_transformations,
    p=1.0,
    bbox_params=A.BboxParams(format="pascal_voc", label_fields=["class_labels"]),
)

# Transformations for validations & test dataset
val_tfms = A.Compose(
    valid_transformations,
    p=1.0,
    bbox_params=A.BboxParams(format="pascal_voc", label_fields=["class_labels"]),
)

**Create `pl.LightningModule` instance :** 

In [ ]:
class DetectionModel(pl.LightningModule):
    def __init__(self,model: nn.Module, hparams: argparse.Namespace) -> None:
        super(DetectionModel, self).__init__()
        self.model = model # model
        self.hparams = hparams # hyperparameters
        
    @property
    def num_batches(self) -> List[int]:
        "returns the number of batches in train, validaiton & test dataloader"
        return [len(self.hparams.train_dl), len(self.hparams.val_dl), len(self.hparams.test_dl)]
    
    # ---------------------------------------------------------------- #
    # Configure Optimizer & Scheduler for the model
    # ---------------------------------------------------------------- #
    def configure_optimizers(self, *args, **kwargs):
        # optimizer
        self.optimizer = self.hparams.optimizer
        # stepLrScheduler
        self.scheduler = self.hparams.scheduler
        return [self.optimizer], [self.scheduler]

    # ---------------------------------------------------------------- #
    # Train Logic:
    # ---------------------------------------------------------------- #
    def train_dataloader(self, *args, **kwargs):
        return self.hparams.train_dl

    def forward(self, xb, *args, **kwargs):
        return self.model(xb)

    def training_step(self, batch, batch_idx, *args, **kwargs):
        images, targets, _ = batch
        targets = [{k: v for k, v in t.items()} for t in targets]
        loss_dict = self.model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        return {"loss": losses, "log": loss_dict, "progress_bar": loss_dict}

    # ---------------------------------------------------------------- #
    # Validation Logic:
    # ---------------------------------------------------------------- #
    def val_dataloader(self, *args, **kwargs):
        loader = self.hparams.val_dl
        coco = get_coco_api_from_dataset(loader.dataset)
        self.coco_evaluator = CocoEvaluator(coco, self.hparams.iou_types)
        return loader

    def validation_step(self, batch, batch_idx, *args, **kwargs):
        images, targets, _ = batch
        targets = [{k: v for k, v in t.items()} for t in targets]
        outputs = self.model(images, targets)
        res = {target["image_id"].item(): output for target, output in zip(targets, outputs)}
        self.coco_evaluator.update(res)
        return {}

    def validation_epoch_end(self, outputs, *args, **kwargs):
        self.coco_evaluator.accumulate()
        self.coco_evaluator.summarize()
        metric = self.coco_evaluator.coco_eval["bbox"].stats[0]
        metric = torch.as_tensor(metric)
        logs = {"COCO_mAP": metric}
        return {"val_loss": metric, "log": logs,"progress_bar": logs,}
    
    # ---------------------------------------------------------------- #
    # Test Logic:
    # ---------------------------------------------------------------- #
    def test_dataloader(self, *args, **kwargs):
        loader = self.hparams.test_dl
        coco = get_coco_api_from_dataset(loader.dataset)
        self.test_evaluator = CocoEvaluator(coco, self.hparams.iou_types)
        return loader

    def test_step(self, batch, batch_idx, *args, **kwargs):
        images, targets, _ = batch
        targets = [{k: v for k, v in t.items()} for t in targets]
        outputs = self.model(images, targets)
        res = {target["image_id"].item(): output for target, output in zip(targets, outputs)}
        self.test_evaluator.update(res)
        return {}
    
    def test_epoch_end(self, outputs, *args, **kwargs):
        self.test_evaluator.accumulate()
        self.test_evaluator.summarize()
        metric = self.test_evaluator.coco_eval["bbox"].stats[0]
        metric = torch.as_tensor(metric)
        logs = {"COCO_mAP": metric}
        return {"COCO_mAP": metric, "log": logs, "progress_bar": logs,}

**Some Helper Functions :**

In [ ]:
def get_dataloaders(
    trn_df: pd.DataFrame, # train DataFrame
    val_df: pd.DataFrame, # valid DataFrame
    tst_df: pd.DataFrame, # test DataFrame
    trn_tfms: A.Compose, # albumentations transformations for train data
    val_tfms: A.Compose, # albumentations transformations for validation & test data
    trn_bs: int, # train batch_size
    val_bs: int, # validation & test batch_size
    ) -> Tuple[DataLoader, DataLoader, DataLoader]:

    "Returns DataLoaders from given dataframes"
    # Instatiate the Detections Datasets
    trn_ds = DetectionDataset(trn_df, trn_tfms) # train dataset
    val_ds = DetectionDataset(val_df, val_tfms) # validaition dataset
    tst_ds = DetectionDataset(tst_df, val_tfms) # test dataset
    
    # Train Dataloader 
    trn_dl = DataLoader(trn_ds, batch_size=trn_bs, shuffle=True, collate_fn=collate_fn, pin_memory=True,)
    # Validation Dataloader
    val_dl = DataLoader(val_ds,batch_size=val_bs, shuffle=False, collate_fn=collate_fn, pin_memory=False,)
    # Test Dataloader
    tst_dl = DataLoader(tst_ds, batch_size=val_bs, shuffle=False, collate_fn=collate_fn, pin_memory=False)
    # return dataloaders
    return trn_dl, val_dl, tst_dl

In [ ]:
def get_model(
    lr: float, # learning_rate
    num_epochs: int, # total number of epochs
    nc: int, # number of classes
    df_train: Union[pd.DataFrame, None] = None, # train DataFrame
    df_val: Union[pd.DataFrame, None] = None, # Validaiton DataFrame
    df_test: Union[pd.DataFrame, None] = None, # Test DataFrame
    trn_tfms: Union[A.Compose, None] = None, # albumentations transformations for train data
    val_tfms: Union[A.Compose, None] = None, # albumentations transformations for validation & test data
    trn_bs: Union[int, None] = None, # train batch_size
    val_bs: Union[int, None] = None, # validation & test batch_size
    bkb: Union[str, None] = None, # name of the resnet based backbone
) -> Tuple[pl.LightningModule, List[DataLoader], argparse.Namespace]:

    """
    Creates lightning Module instance from given arguments

    Returns:
        1. pl_model (pl.LightningModule)  : a pl.LightningModule instance
        2. dataloaders (List[Dataloader]) : list of train, val, test dataloaders
        3. hparams ([argparse.Namespace]) : hyperparmeters
    """
    # Instantiate RetinaNet Model
    model = Retinanet(num_classes=nc, backbone_kind=bkb)
    
    # Instantiate in the DataLoaders
    train_dl, val_dl, test_dl = get_dataloaders(df_train, df_val, df_test, trn_tfms, val_tfms, trn_bs, val_bs)
    
    # Instatiate Optimizer
    params = [p for p in model.parameters() if p.requires_grad] # trainable parameters
    optimizer = optim.AdamW(params, lr, weight_decay=1e-02) # optimizer
    
    # Instantiate Scheduler
    scheduler = {
        "scheduler": optim.lr_scheduler.OneCycleLR(optimizer, lr, epochs=num_epochs, steps_per_epoch=len(train_dl)),
        "interval": "step", # [one of step or epoch]
        "frequency": 1,
        }

    # Create Config Dictionary:
    conf_dict = {
        "train_dl": train_dl, # train dataloader
        "val_dl": val_dl,  # validation dataloader
        "test_dl": test_dl, # test dataloader
        "iou_types": ["bbox"], # for Ivaluation
        "optimizer": optimizer, # optimizer
        "scheduler": scheduler, # scheduler
        } 
    
    # Convert config dictionary to `argparse.Namespace` instance
    hparams = argparse.Namespace(**conf_dict)

    # Instantiate lightning module
    pl_model = DetectionModel(model, hparams)
    dataloaders = [train_dl, val_dl, test_dl] 
    
    return pl_model, dataloaders, hparams

In [ ]:
def get_trainer(filepath: Union[str, None] = None, **kwargs) -> pl.Trainer:
    "Returns a pl.Trainer instance"
    
    if filepath is None:
        filepath = "/content/drive/My Drive/pascal_checkpoints" 
    os.makedirs(filepath, exist_ok=True)

    # Wandb logger: assuming wandb is set-up [Optional]
    wb_logger = WandbLogger(
        name=f"retinanet-pets-{time.strftime('%d-%m:::%I.%M.%S%p')}",
        project="pascal-2007",
        anonymous="allow",
    )

    # Learning_rate logger to monitor learning_rate [Optional]
    lr_logger = LearningRateLogger(logging_interval="step")

    # checkpoint callback
    checkpoint_callback = ModelCheckpoint(
        filepath=filepath,
        mode="max",
        monitor="COCO_mAP",
        save_top_k=1,
        save_weights_only=True,
    )

    # early stopping callback
    early_stopping_callback = EarlyStopping(mode="max", monitor="COCO_mAP", patience=5,)

    # Trainer
    trainer = pl.Trainer(
        logger=[wb_logger], # wandb logger
        callbacks=[lr_logger], # callback to log the learning rate to wandb
        num_sanity_val_steps=0, # no need to do sanity check
        benchmark=True, # squeeze extra performance from the GPU
        early_stop_callback=early_stopping_callback, # early stopping callback
        checkpoint_callback=checkpoint_callback, # checkpoints the highest mAP model
        terminate_on_nan=True, # Terminate if values become `nan`
        **kwargs,
    )

    return trainer

**Set Training Parameter sand Grab the `model` & `trainer`:**

In [ ]:
# ------------------------------- #
# Training Parameters :
# ------------------------------- #
TRAIN_BATCH_SIZE = 32 # Batch size for train dataset
VALID_BATCH_SIZE = 32 # batch size for valid & test dataset
LR = 4e-03 # learning_rate for Optimizer
NUM_CLASSES = 20  # Pascal 2007 has 20 Classes
NUM_EPOCHS = 30

# Instantiate the model
retinanet, dataloaders, conf_dict = get_model(
    LR, # learning rate
    NUM_EPOCHS, # total number of epochs
    NUM_CLASSES, # total number of unique target classes
    trn_df, # train dataframe
    val_df, # validation dataframe
    tst_df, # test dataframe
    trn_tfms, # train transformations [albumentations]
    val_tfms, # valid & test dataframe [albumentations]
    trn_bs=TRAIN_BATCH_SIZE, # train batch_size
    val_bs=VALID_BATCH_SIZE, # valid & test batch_size
    bkb="resnet50", # kind of resnet backbone
)

# Instantiate Trainer
trainer = get_trainer(check_val_every_n_epoch=5, gpus=1, precision=16, gradient_clip_val=0.1, max_epochs=NUM_EPOCHS,)

**Train model:**

In [ ]:
# Fit Model 
trainer.fit(retinanet)

**Evaluate:**

In [ ]:
# Evaluate model on test dataloadet
trainer.test()